# Feature Importance

In [1]:
!pip install sqlalchemy
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.0 MB/s eta 0:00:00


In [7]:
import gc, sqlite3
import pandas as pd, numpy as np
from catboost import CatBoostRanker, Pool

import sys
if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive")

_DB_PATH = "/content/drive/MyDrive/Colab Notebooks/database.db"
TEST_IDS = "test_ids.csv"
VAL_IDS  = "val_ids.csv"
DROP = ["label","clean_row_id","investor","firm","template_id"]

# IDs
val_ids  = pd.read_csv(VAL_IDS)["val_ids"].dropna().astype(int).tolist()
test_ids = pd.read_csv(TEST_IDS)["test_ids"].dropna().astype(int).tolist()

# Data
chunks = []
with sqlite3.connect(_DB_PATH) as conn:
    for ch in pd.read_sql_query("SELECT * FROM feature_matrix", conn, chunksize=100_000):
        chunks.append(ch)
full_df = pd.concat(chunks, ignore_index=True)

# Split
val_ids_set  = set(val_ids)
test_ids_set = set(test_ids)
excluded_ids = val_ids_set | test_ids_set

val_df  = full_df[full_df["clean_row_id"].isin(val_ids_set)].reset_index(drop=True)
train_df = full_df[~full_df["clean_row_id"].isin(excluded_ids)].reset_index(drop=True)

# Feature columns
feature_cols = [c for c in train_df.columns if c not in DROP]

def build_pool(df):
    sizes = df.groupby("clean_row_id", sort=False).size().tolist()
    gid = np.repeat(np.arange(len(sizes)), sizes)
    X = df[feature_cols].copy()
    y = df["label"].astype(int)
    for c in X.columns:
        if X[c].dtype == bool:
            X[c] = X[c].astype(np.int8)
    return Pool(data=X, label=y, group_id=gid, feature_names=feature_cols)

train_pool = build_pool(train_df)
val_pool   = build_pool(val_df)

# Load trained model
model = CatBoostRanker()
model.load_model("syn_catboost_model.cbm")

# CatBoost feature importance (PredictionValuesChange)
fi_vals = model.get_feature_importance(train_pool, type="PredictionValuesChange")
fi = (pd.DataFrame({"feature": feature_cols, "importance": fi_vals})
        .sort_values("importance", ascending=False))
print("Top CatBoost importances:\n", fi.head(10), "\n")

# SHAP (mean absolute across validation)
shap_matrix = model.get_feature_importance(data=val_pool, type="ShapValues")
# last column is the base value
shap_mean_abs = np.abs(shap_matrix[:, :-1]).mean(axis=0)
shap_df = (pd.DataFrame({"feature": feature_cols, "mean_abs_shap": shap_mean_abs})
             .sort_values("mean_abs_shap", ascending=False))
print("Top SHAP (mean |SHAP|):\n", shap_df.head(10))

# Save for Appendix
fi.to_csv("catboost_importance.csv", index=False)
shap_df.to_csv("catboost_shap_meanabs.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Top CatBoost importances:
                                feature  importance
20          template_firm_coverage_pct   37.656567
21            template_is_top_template   10.895586
10              template_support_count    6.975788
0           template_in_firm_templates    6.688859
19         template_firm_support_count    4.740155
11               template_coverage_pct    4.584893
16       template_uses_multiple_firsts    3.983525
24                  firm_num_investors    3.546723
18        template_uses_multiple_lasts    2.792086
22  template_name_characteristic_clash    2.447118 

Top SHAP (mean |SHAP|):
                          feature  mean_abs_shap
20    template_firm_coverage_pct       0.078463
24            firm_num_investors       0.058747
26       firm_is_single_template       0.043843
10        template_support_count       0.037708
5          inves